In [1]:
import re
import pandas as pd
import numpy as np
import seaborn as sns

## Data cleaning 

In [6]:
# Load data
df = pd.read_csv('datasets/scraped.csv')
df

,code,number,price,city
0,3,344,"229,000 AED",uae_sharjah
1,U,1313,"109,000 AED",duabi_new
2,H,3222,"72,000 AED",duabi_new
3,3,4441,"38,000 AED",uae_sharjah
4,3,7776,"38,000 AED",uae_sharjah
...,...,...,...,...
2741,X,81686,"AED2,000",duabi_new
2742,X,91393,"AED2,000",duabi_new
2743,X,91494,"AED2,000",duabi_new
2744,X,91595,"AED2,000",duabi_new


,unnamed: 0,code,number,price,city
0,0,3,344,"229,000 AED",uae_sharjah
1,1,U,1313,"109,000 AED",duabi_new
2,2,H,3222,"72,000 AED",duabi_new
3,3,3,4441,"38,000 AED",uae_sharjah
4,4,3,7776,"38,000 AED",uae_sharjah
...,...,...,...,...,...
2741,2741,X,81686,"AED2,000",duabi_new
2742,2742,X,91393,"AED2,000",duabi_new
2743,2743,X,91494,"AED2,000",duabi_new
2744,2744,X,91595,"AED2,000",duabi_new


In [5]:
# Remove AED and commans from price
plates['Price'] = plates['Price'].apply(lambda x: re.sub(r"[a-zA-Z,]",'',x)).astype(int)

NameError: name 'plates' is not defined